In [1]:
import random
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageEnhance

In [6]:
path = 'images/B_training_set/'
destination = 'images/D_augmented_set'

def brighten(image):
  enhancer = ImageEnhance.Brightness(image)
  return enhancer.enhance(random.randint(10, 25)/10), True

def flip(image):
  new_image = ImageOps.flip(image)
  return ImageOps.mirror(new_image), True

def rotate(image):
  return image.rotate(random.randint(1, 359)), True

def gaussian_noise(image):
  new_image = np.asarray(image)
  noise = np.random.normal(loc=1.0, scale=random.randint(100, 400)/1000, size=new_image.shape)
  new_image = new_image * noise
  new_image = new_image.astype(np.uint8).clip(0, 255)
  return new_image, False

def save(image, new_image, boolean, n):
  if boolean:
    new_image = new_image.save(f'{destination}/{image}-{n}.jpg')
  else:
    plt.imsave(f'{destination}/{image}-{n}.jpg', new_image)

In [7]:
for image in os.listdir(path):
  image = Image.open(path + image)
# (1) FLIP + BRIGHTNESS 
  new_image, boolean = flip(image)
  new_image, boolean = brighten(new_image)
  save(image, new_image, boolean, 1)
  # (2) FLIP + ROTATION
  new_image, boolean = flip(image)
  new_image, boolean = rotate(new_image)
  save(image, new_image, boolean, 2)
  # (3) FLIP + GAUSSIAN NOISE
  new_image, boolean = flip(image)
  new_image, boolean = gaussian_noise(new_image)
  save(image, new_image, boolean, 3)
  # (4) BRIGHTNESS + ROTATION
  new_image, boolean = brighten(image)
  new_image, boolean = rotate(new_image)
  save(image, new_image, boolean, 4)
  # (5) BRIGHTNESS + GAUSSIAN NOISE
  new_image, boolean = brighten(image)
  new_image, boolean = gaussian_noise(new_image)
  save(image, new_image, boolean, 5)
  # (6) ROTATION + GAUSSIAN NOISE
  new_image, boolean = rotate(image)
  new_image, boolean = gaussian_noise(new_image)
  save(image, new_image, boolean, 6)
  # (7) FLIP + BRIGHTNESS + ROTATION
  new_image, boolean = flip(image)
  new_image, boolean = brighten(new_image)
  new_image, boolean = rotate(new_image)
  save(image, new_image, boolean, 7)
  # (8) FLIP + ROTATION + GAUSSIAN NOISE
  new_image, boolean = flip(image)
  new_image, boolean = rotate(new_image)
  new_image, boolean = gaussian_noise(new_image)
  save(image, new_image, boolean, 8)
  # (9) FLIP + BRIGHTNESS + GAUSSIAN NOISE
  new_image, boolean = flip(image)
  new_image, boolean = brighten(new_image)
  new_image, boolean = gaussian_noise(new_image)
  save(image, new_image, boolean, 9)
  # (10) BRIGHTNESS + ROTATION + GAUSSIAN NOISE
  new_image, boolean = brighten(image)
  new_image, boolean = rotate(new_image)
  new_image, boolean = gaussian_noise(new_image)
  save(image, new_image, boolean, 10)

KeyboardInterrupt: 

In [5]:
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
from fastai.vision.all import *
path = '/Users/limenxi/Documents/vegetables'
def label_func(f): return f[0].isupper()
files3 = get_image_files(destination)
len(files3)

1000

In [6]:
dls3 = ImageDataLoaders.from_name_func(path, files3, label_func, item_tfms=Resize(480))
learn3 = vision_learner(dls3, resnet34, metrics=accuracy)
learn3.fine_tune(5)

/Users/limenxi/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/limenxi/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,accuracy,time
0,0.982953,0.455511,0.795000,21:23


epoch,train_loss,valid_loss,accuracy,time
0,0.363253,0.236908,0.890000,34:01
1,0.269065,0.218106,0.935000,30:04
2,0.202644,0.163080,0.925000,31:00
3,0.150872,0.119994,0.955000,31:33
4,0.114950,0.118303,0.960000,1:22:16


In [7]:
test = '/Users/limenxi/Documents/vegetables/A_test_set/'
results = 0
healthy = 0

for image in os.scandir(test):
  if str(image)[11].isupper() and learn3.predict(test + str(image)[11:-2])[0] == 'True':
    results += 1
    healthy += 1
  elif str(image)[11].islower() and learn3.predict(test + str(image)[11:-2])[0] == 'False':
    results += 1

print(f'Overall accuracy: {results}%')
print(f'Healthy accuracy: {healthy * 2}%')
print(f'Rusted accuracy: {(results - healthy) * 2}%')

Overall accuracy: 96%
Healthy accuracy: 94%
Rusted accuracy: 98%
